# ECSE 420 - Lab 3
Drum hit simulation using a grid of finite elements.

In [0]:
import os
dev_lib_path = !find / -iname 'libdevice'
nvvm_lib_path = !find / -iname 'libnvvm.so'
assert len(dev_lib_path)>0, "Device Lib Missing"
assert len(nvvm_lib_path)>0, "NVVM Missing"
os.environ['NUMBAPRO_LIBDEVICE'] = dev_lib_path[0]
os.environ['NUMBAPRO_NVVM'] = nvvm_lib_path[0]

In [0]:
from numba import cuda
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

# Question 1 - Sequential Implementation

In [3]:
def sequential(N=4, iterations=4):
    u = np.zeros([N, N], dtype = float)
    u1 = u.copy()
    u2 = u.copy()

    G = 0.75
    rho = 0.5
    nu = 0.0002

    # hit = "1" at u(2,2)
    u1[2][2] = 1

    for itera in range(iterations):
        # response 1 to N-2
        for i in range(1, N-1):
            for j in range(1, N-1):
                u[i][j] = ((rho * (u1[i-1][j] + u1[i+1][j] + 
                                  u1[i][j-1] + u1[i][j+1] - 
                                  4*u1[i][j])) + 
                           (2 * u1[i][j]) - ((1 - nu) * u2[i][j]))
                u[i][j] /= (1 + nu)

        # print("pre edges Iteration: {} \nGrid:\n{}\n".format(itera,u))
        # range 1 to N-2
        for x in range(1, N-1):
            ## EDGES
            # left edge = .75 * interior
            u[0][x] = G * u[1][x]
            # right edge
            u[N-1][x] = G * u[N-2][x]
            # top edge
            u[x][0] = G * u[x][1]
            # bottom edge
            u[x][N-1] = G * u[x][N-2]


        # print("pre corners Iteration: {} \nGrid:\n{}\n".format(itera,u))

        ## CORNERS
        # top left
        u[0][0] = G * u[1][0]
        # top right
        u[N-1][0] = G * u[N-2][0]
        # bottom left
        u[0][N-1] = G * u[0][N-2]
        # bottom right
        u[N-1][N-1] = G * u[N-1][N-2]

        u2 = u1.copy()
        u1 = u.copy()

        print(u[2][2])
        print("Iteration: {} \nGrid:\n{}\n".format(itera,u))


sequential(4, 4)

0.0
Iteration: 0 
Grid:
[[0.         0.         0.37492501 0.28119376]
 [0.         0.         0.49990002 0.37492501]
 [0.37492501 0.49990002 0.         0.        ]
 [0.28119376 0.37492501 0.         0.        ]]

-0.49980001999999923
Iteration: 1 
Grid:
[[ 0.28113753  0.37485004  0.28113753  0.21085315]
 [ 0.37485004  0.49980006  0.37485004  0.28113753]
 [ 0.28113753  0.37485004 -0.49980002 -0.37485001]
 [ 0.21085315  0.28113753 -0.37485001 -0.28113751]]

2.9982007234403314e-08
Iteration: 2 
Grid:
[[ 4.21621976e-01  5.62162635e-01 -1.63964072e-01 -1.22973054e-01]
 [ 5.62162635e-01  7.49550180e-01 -2.18618763e-01 -1.63964072e-01]
 [-1.63964072e-01 -2.18618763e-01  2.99820072e-08  2.24865054e-08]
 [-1.22973054e-01 -1.63964072e-01  2.24865054e-08  1.68648791e-08]]

0.28102511495301846
Iteration: 3 
Grid:
[[-0.08782031 -0.11709375 -0.12294844 -0.09221133]
 [-0.11709375 -0.156125   -0.16393126 -0.12294844]
 [-0.12294844 -0.16393126  0.28102511  0.21076884]
 [-0.09221133 -0.12294844  0.2107

# Question 2 - Parallel Implementation #1
One thread per finite element of the grid

In [4]:
@cuda.jit
def drum_interior(grid, grid_back1, grid_back2, n, p):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < 16:
    j = idx // 4
    i = idx % 4

    if i not in [0, 3] and j not in [0, 3]:
      # interior node
      grid[i][j] = ((p * (grid_back1[i-1][j] + grid_back1[i+1][j] + 
                          grid_back1[i][j-1] + grid_back1[i][j+1] - 
                          4 * grid_back1[i][j])) + 
                    (2 * grid_back1[i][j]) - ((1 - n) * grid_back2[i][j]))
      grid[i][j] /= (1 + n)

@cuda.jit
def drum_edges(grid, G):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < 16:
    j = idx // 4
    i = idx % 4

    if i == 0 and j in [1, 2]:
      # left edge
      grid[i][j] = G * grid[i + 1][j]
    elif i == 3 and j in [1, 2]:
      # right edge
      grid[i][j] = G * grid[i - 1][j]
    elif j == 0 and i in [1, 2]:
      # top edge
      grid[i][j] = G * grid[i][j + 1]
    elif j == 3 and i in [1, 2]:
      # bottom edge
      grid[i][j] = G * grid[i][j - 1]

@cuda.jit
def drum_corners(grid, G):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < 16:
    j = idx // 4
    i = idx % 4

    if i == 0 and j == 0:
      # top left corner
      grid[i][j] = G * grid[i + 1][j]
    elif i == 0 and j == 3:
      # bottom left corner
      grid[i][j] = G * grid[i][j - 1]
    elif i == 3 and j == 0:
      # top right corner
      grid[i][j] = G * grid[i][j + 1]
    elif i == 3 and j == 3:
      # bottom right corner
      grid[i][j] = G * grid[i][j - 1]

# constant parameters
G = 0.75
n = 0.0002
p = 0.5
gridsize = 4

# grid creation
grid = np.zeros((gridsize, gridsize))
grid_back1 = grid.copy()
grid_back2 = grid.copy()
grid_back1[2][2] = 1   # simulate drum hit at time t-1

# variable parameters
numBlocks = 978
threadsPerBlock = 1000
T = 50

# simulate time progression
for t in range(T):
  # function call (numba performs host -> device -> host memory transfers)
    # required order: interior -> edges -> corners
  drum_interior[numBlocks, threadsPerBlock](grid, grid_back1, grid_back2, n, p)
  drum_edges[numBlocks, threadsPerBlock](grid, G)
  drum_corners[numBlocks, threadsPerBlock](grid, G)
  
  print("u[2, 2] at iteration {}: {}".format(t + 1, grid[2][2]))

  if (t < 3):
    print(grid)
    print()

  grid_back2 = grid_back1.copy()
  grid_back1 = grid.copy()

u[2, 2] at iteration 1: 0.0
[[0.         0.         0.37492501 0.28119376]
 [0.         0.         0.49990002 0.37492501]
 [0.37492501 0.49990002 0.         0.        ]
 [0.28119376 0.37492501 0.         0.        ]]

u[2, 2] at iteration 2: -0.49980001999999923
[[ 0.28113753  0.37485004  0.28113753  0.21085315]
 [ 0.37485004  0.49980006  0.37485004  0.28113753]
 [ 0.28113753  0.37485004 -0.49980002 -0.37485001]
 [ 0.21085315  0.28113753 -0.37485001 -0.28113751]]

u[2, 2] at iteration 3: 2.9982007234403314e-08
[[ 4.21621976e-01  5.62162635e-01 -1.63964072e-01 -1.22973054e-01]
 [ 5.62162635e-01  7.49550180e-01 -2.18618763e-01 -1.63964072e-01]
 [-1.63964072e-01 -2.18618763e-01  2.99820072e-08  2.24865054e-08]
 [-1.22973054e-01 -1.63964072e-01  2.24865054e-08  1.68648791e-08]]

u[2, 2] at iteration 4: 0.2810251149530184
u[2, 2] at iteration 5: 0.04682818495502672
u[2, 2] at iteration 6: -0.08778516995465212
u[2, 2] at iteration 7: -0.32181476130245723
u[2, 2] at iteration 8: -0.7413666495

# Question 3 - Parallel Implementation #3
Variable number of finite elements per thread. Output at [N/2, N/2] for 50 iterations is shown below (for comparison with posted file).

In [5]:
@cuda.jit
def drum_interiorMult(grid, grid_back1, grid_back2, n, p, gridsize, elementsPerThread, threadsRequired):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < threadsRequired:
    for iteration in range(elementsPerThread):
      # updating on adjacent nodes isn't a problem because we're using the history grids only
      realIdx = idx * elementsPerThread + iteration

      if realIdx < gridsize * gridsize:
        j = realIdx // gridsize
        i = realIdx % gridsize

        if i != 0 and j != 0 and i != gridsize - 1 and j != gridsize - 1:
          # interior node
          grid[i][j] = ((p * (grid_back1[i-1][j] + grid_back1[i+1][j] + 
                              grid_back1[i][j-1] + grid_back1[i][j+1] - 
                              4 * grid_back1[i][j])) + 
                        (2 * grid_back1[i][j]) - ((1 - n) * grid_back2[i][j]))
          grid[i][j] /= (1 + n)

@cuda.jit
def drum_edgesMult(grid, G, gridsize, elementsPerThread, threadsRequired):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < threadsRequired:
    for iteration in range(elementsPerThread):
      # updating on adjacent nodes isn't a problem because we're using the history grids only
      realIdx = idx * elementsPerThread + iteration

      if realIdx < gridsize * gridsize:
        j = realIdx // gridsize
        i = realIdx % gridsize

        if i == 0 and j in [1, 2]:
          # left edge
          grid[i][j] = G * grid[i + 1][j]
        elif i == 3 and j in [1, 2]:
          # right edge
          grid[i][j] = G * grid[i - 1][j]
        elif j == 0 and i in [1, 2]:
          # top edge
          grid[i][j] = G * grid[i][j + 1]
        elif j == 3 and i in [1, 2]:
          # bottom edge
          grid[i][j] = G * grid[i][j - 1]

@cuda.jit
def drum_cornersMult(grid, G, gridsize, elementsPerThread, threadsRequired):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < threadsRequired:
    for iteration in range(elementsPerThread):
      # updating on adjacent nodes isn't a problem because we're using the history grids only
      realIdx = idx * elementsPerThread + iteration

      if realIdx < gridsize * gridsize:
        j = realIdx // gridsize
        i = realIdx % gridsize

        if i == 0 and j == 0:
          # top left corner
          grid[i][j] = G * grid[i + 1][j]
        elif i == 0 and j == 3:
          # bottom left corner
          grid[i][j] = G * grid[i][j - 1]
        elif i == 3 and j == 0:
          # top right corner
          grid[i][j] = G * grid[i][j + 1]
        elif i == 3 and j == 3:
          # bottom right corner
          grid[i][j] = G * grid[i][j - 1]

# constant parameters
G = 0.75
n = 0.0002
p = 0.5
gridsize = 512

# grid creation
grid = np.zeros((gridsize, gridsize))
grid_back1 = grid.copy()
grid_back2 = grid.copy()
grid_back1[gridsize // 2][gridsize // 2] = 1   # simulate drum hit at time t-1

# variable parameters
numBlocks = 978
threadsPerBlock = 1000
elementsPerThread = 16
T = 50

threadsRequired = math.ceil((gridsize * gridsize) / elementsPerThread)

# simulate time progression
for t in range(T):
  # function call (numba performs host -> device -> host memory transfers)
    # required order: interior -> edges -> corners
  drum_interiorMult[numBlocks, threadsPerBlock](grid, grid_back1, grid_back2, n, p, gridsize, elementsPerThread, threadsRequired)
  drum_edgesMult[numBlocks, threadsPerBlock](grid, G, gridsize, elementsPerThread, threadsRequired)
  drum_cornersMult[numBlocks, threadsPerBlock](grid, G, gridsize, elementsPerThread, threadsRequired)
  
  print("Iteration {}: {}".format(t + 1, grid[gridsize // 2][gridsize // 2]))

  grid_back2 = grid_back1.copy()
  grid_back1 = grid.copy()

Iteration 1: 0.0
Iteration 2: 3.998400479824981e-08
Iteration 3: 0.0
Iteration 4: 0.24980013992803154
Iteration 5: 0.0
Iteration 6: 8.9892080370116e-08
Iteration 7: 0.0
Iteration 8: 0.14040029222121936
Iteration 9: 0.0
Iteration 10: 1.403440891682952e-07
Iteration 11: 0.0
Iteration 12: 0.09742231969133452
Iteration 13: 0.0
Iteration 14: 1.9087120709162718e-07
Iteration 15: 0.0
Iteration 16: 0.0745294056393437
Iteration 17: 0.0
Iteration 18: 2.413783011458448e-07
Iteration 19: 0.0
Iteration 20: 0.060320634512427584
Iteration 21: 0.0
Iteration 22: 2.918343471751079e-07
Iteration 23: 0.0
Iteration 24: 0.050645649253699805
Iteration 25: 0.0
Iteration 26: 3.4222637009984303e-07
Iteration 27: 0.0
Iteration 28: 0.0436341257299501
Iteration 29: 0.0
Iteration 30: 3.925480441346873e-07
Iteration 31: 0.0
Iteration 32: 0.03831973287361857
Iteration 33: 0.0
Iteration 34: 4.4279595196121775e-07
Iteration 35: 0.0
Iteration 36: 0.03415301791490642
Iteration 37: 0.0
Iteration 38: 4.929681108493421e-07


### Comparison with Previous Implementations
Compare implementation #3 with implementations #1 and #2 to ensure that results match.

In [6]:
# constant parameters
G = 0.75
n = 0.0002
p = 0.5
gridsize = 4

# grid creation
grid = np.zeros((gridsize, gridsize))
grid_back1 = grid.copy()
grid_back2 = grid.copy()
grid_back1[gridsize // 2][gridsize // 2] = 1   # simulate drum hit at time t-1

# variable parameters
numBlocks = 978
threadsPerBlock = 1000
elementsPerThread = 16
T = 50

threadsRequired = math.ceil((gridsize * gridsize) / elementsPerThread)

# simulate time progression
for t in range(T):
  # function call (numba performs host -> device -> host memory transfers)
    # required order: interior -> edges -> corners
  drum_interiorMult[numBlocks, threadsPerBlock](grid, grid_back1, grid_back2, n, p, gridsize, elementsPerThread, threadsRequired)
  drum_edgesMult[numBlocks, threadsPerBlock](grid, G, gridsize, elementsPerThread, threadsRequired)
  drum_cornersMult[numBlocks, threadsPerBlock](grid, G, gridsize, elementsPerThread, threadsRequired)
  
  print("Iteration {}: {}".format(t + 1, grid[gridsize // 2][gridsize // 2]))

  if (t < 3):
    print(grid)

  grid_back2 = grid_back1.copy()
  grid_back1 = grid.copy()

Iteration 1: 0.0
[[0.         0.         0.37492501 0.28119376]
 [0.         0.         0.49990002 0.37492501]
 [0.37492501 0.49990002 0.         0.        ]
 [0.28119376 0.37492501 0.         0.        ]]
Iteration 2: -0.49980001999999923
[[ 0.28113753  0.37485004  0.28113753  0.21085315]
 [ 0.37485004  0.49980006  0.37485004  0.28113753]
 [ 0.28113753  0.37485004 -0.49980002 -0.37485001]
 [ 0.21085315  0.28113753 -0.37485001 -0.28113751]]
Iteration 3: 2.9982007234403314e-08
[[ 4.21621976e-01  5.62162635e-01 -1.63964072e-01 -1.22973054e-01]
 [ 5.62162635e-01  7.49550180e-01 -2.18618763e-01 -1.63964072e-01]
 [-1.63964072e-01 -2.18618763e-01  2.99820072e-08  2.24865054e-08]
 [-1.22973054e-01 -1.63964072e-01  2.24865054e-08  1.68648791e-08]]
Iteration 4: 0.2810251149530184
Iteration 5: 0.04682818495502672
Iteration 6: -0.08778516995465212
Iteration 7: -0.32181476130245723
Iteration 8: -0.74136664956306
Iteration 9: -0.38839948920730266
Iteration 10: 0.6652254330040654
Iteration 11: 0.778

### Performance Measurements

In [7]:
import time

# constant parameters
G = 0.75
n = 0.0002
p = 0.5
gridsize = 512
T = 500

# grid creation
grid = np.zeros((gridsize, gridsize))
grid_back1 = grid.copy()
grid_back2 = grid.copy()
grid_back1[gridsize // 2][gridsize // 2] = 1   # simulate drum hit at time t-1

for numBlocks in [1, 4, 8, 32, 128, 512, 2048]:
  for threadsPerBlock in [1, 4, 16, 64, 256, 1024]:
    for elementsPerThread in [1, 4, 16, 64, 256]:
      start = time.time()
      threadsRequired = math.ceil((gridsize * gridsize) / elementsPerThread)

      # simulate time progression
      for t in range(T):
        # function call (numba performs host -> device -> host memory transfers)
          # required order: interior -> edges -> corners
        drum_interiorMult[numBlocks, threadsPerBlock](grid, grid_back1, grid_back2, n, p, gridsize, elementsPerThread, threadsRequired)
        drum_edgesMult[numBlocks, threadsPerBlock](grid, G, gridsize, elementsPerThread, threadsRequired)
        drum_cornersMult[numBlocks, threadsPerBlock](grid, G, gridsize, elementsPerThread, threadsRequired)

        grid_back2 = grid_back1.copy()
        grid_back1 = grid.copy()
      
      print("Runtime for {} blocks, {} threads, {} elements: {}".format(numBlocks, threadsPerBlock, elementsPerThread, time.time() - start))

Runtime for 1 blocks, 1 threads, 1 elements: 4.120593786239624
Runtime for 1 blocks, 1 threads, 4 elements: 4.085301876068115
Runtime for 1 blocks, 1 threads, 16 elements: 4.075458288192749
Runtime for 1 blocks, 1 threads, 64 elements: 4.0888471603393555
Runtime for 1 blocks, 1 threads, 256 elements: 4.232138156890869
Runtime for 1 blocks, 4 threads, 1 elements: 4.0656092166900635
Runtime for 1 blocks, 4 threads, 4 elements: 4.0501868724823
Runtime for 1 blocks, 4 threads, 16 elements: 4.025616645812988
Runtime for 1 blocks, 4 threads, 64 elements: 4.126385927200317
Runtime for 1 blocks, 4 threads, 256 elements: 4.3583338260650635
Runtime for 1 blocks, 16 threads, 1 elements: 4.064435958862305
Runtime for 1 blocks, 16 threads, 4 elements: 4.055553197860718
Runtime for 1 blocks, 16 threads, 16 elements: 4.0937628746032715
Runtime for 1 blocks, 16 threads, 64 elements: 4.104035139083862
Runtime for 1 blocks, 16 threads, 256 elements: 4.379432678222656
Runtime for 1 blocks, 64 threads, 1 